In [1]:
# Load dependencies.
import pandas as pd
import numpy as np
module_path = os.path.abspath(os.path.join('/home/renatochaz/git/economic_policy_uncertainty/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
## Load datasets.
ds = pd.read_csv(f'{module_path}/raw-data/py_create_age_epu.csv')
epu = pd.read_csv(f'{module_path}/raw-data/yearly_epu.csv')
cvm = pd.read_csv(f'{module_path}/raw-data/dt_reg_info.csv')


In [3]:
## Set year index to EPU information
epu['ano'] = list(range(2010, 2020, 1))

In [4]:
## Transform cv dt_reg to year format.
cvm['dt_reg'] = pd.to_datetime(cvm['dt_reg']).dt.to_period('Y')

In [5]:
## Transform EPU and CVM in a dict.
epu_dict = dict(zip(epu['ano'], epu['x']))
cvm_dict = dict(zip(cvm['id_cvm'], cvm['dt_reg'].dt.strftime('%Y')))

In [6]:
## Add EPU to dataset.
ds['epu'] = ds['ano'].map(epu_dict)

In [7]:
## Add cvm reg_date.
ds['cvm_reg'] = ds['id_cvm'].map(cvm_dict)

In [8]:
## Finding firms without id_cvm in the dict.
ds.loc[ds['cvm_reg'].isnull(),:]['codigo'].unique()

array(['BALM4', 'BAUH4', 'BDLL4', 'BRKM5', 'CAMB3', 'CBEE3', 'CEDO4',
       'CESP6', 'CLSC4', 'CMIG4', 'CRUZ3', 'CSNA3', 'CTKA4', 'CTNM4',
       'CTSA4', 'CZLT33', 'DAGB33', 'DOHL4', 'EALT4', 'EEEL4', 'ELEK3',
       'ELET3', 'ETER3', 'EUCA4', 'FESA4', 'FRAS3', 'GGBR4', 'GOAU4',
       'GUAR3', 'HOOT4', 'INEP4', 'JFEN3', 'KEPL3', 'LAME4', 'LEVE3',
       'LIXC3', 'LREN3', 'LUXM4', 'MEND5', 'MGEL4', 'MNDL3', 'MSPA3',
       'MTSA4', 'NAFG4', 'PATI3', 'PETR4', 'PMAM3', 'PNVL3', 'POMO4',
       'PTNT3', 'RADL3', 'RANI3', 'REDE3', 'ROMI3', 'RPMG3', 'TASA4',
       'TUPY3', 'VALE3', 'VVAR3', 'WEGE3', 'WSON33'], dtype=object)

In [9]:
## Manually created dict with dt_reg dates.
man_dict = {'BALM4':1970, 'BAUH4':1977, 'BDLL4':1969, 'BRKM5':1978,
            'CAMB3':1985, 'CBEE3':1969, 'CEDO4':1969, 'CESP6':1971,
            'CLSC4':1973, 'CMIG4':1971, 'CRUZ3':1969, 'CSNA3':1943,
            'CTKA4': 1971, 'CTNM4':1971, 'CTSA4':1984, 'CZLT33':2005,
            'DAGB33':1998, 'DOHL4': 1937, 'EALT4': 1950, 'EEEL4': 1970,
            'ELEK3':1971, 'ELET3':1996, 'ETER3':1970, 'EUCA4':1969,
            'FESA4':1961, 'FRAS3':1977, 'GGBR4':1980, 'GOAU4':1968,
            'GUAR3':1958, 'HOOT4':1980, 'INEP4':1980, 'JFEN3': 1977,
            'KEPL3':1980, 'LAME4':1977, 'LEVE3':1977, 'LIXC3':1977,
            'LREN3':1977, 'LUXM4':1980, 'MEND5':1977, 'MGEL4':1971,
            'MNDL3':1979, 'MSPA3':1977, 'MTSA4':1971, 'NAFG4':1977,
            'PATI3':1971, 'PETR4':1977, 'PMAM3':1977, 'PNVL3':1977,
            'POMO4':1977, 'PTNT3':2000, 'RADL3':1977, 'RANI3':1977,
            'REDE3':1969, 'ROMI3':1938, 'RPMG3':1970, 'TASA4':1982,
            'TUPY3':1966, 'VALE3': 1970, 'VVAR3':1981, 'WEGE3':1982, 'WSON33':1998}

In [10]:
## Add cvm reg_date from dict2.
ds['cvm_reg2'] = ds['codigo'].map(man_dict)

In [11]:
## Check where information conflict.
np.where(ds['cvm_reg2'].isnull() == ds['cvm_reg'].isnull())


(array([], dtype=int64),)

In [12]:
## Merge columns together.
ds['cvm_reg'].update(ds.pop('cvm_reg2'))

In [13]:
## Checking NA.
ds['cvm_reg'].isnull().sum()

0

In [14]:
## Create firm age.
ds['idade_firma'] = ds['ano'].astype(int) - ds['cvm_reg'].astype(int)

In [15]:
## Checking negative values
np.where(ds['idade_firma'] < 0)

(array([   0,    1, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1517,
        1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525]),)

In [16]:
## Get firms with inconsistency age
ds.loc[ds['idade_firma'] < 0]['codigo'].unique()

array(['ABEV3', 'NTCO3', 'TIMS3'], dtype=object)

In [17]:
## Get median to change cvm_reg from firms with negative age
ds['cvm_reg'].astype(int).median()

1998.0

In [18]:
## Changing cvm_reg for those firms
index = np.where(ds['codigo'].isin(['ABEV3', 'NTCO3', 'TIMS3']))

In [19]:
ds.iloc[index[0], 41]

0       2013
1       2013
2       2013
3       2013
4       2013
5       2013
6       2013
7       2013
8       2013
1067    2019
1068    2019
1069    2019
1070    2019
1071    2019
1072    2019
1073    2019
1074    2019
1075    2019
1517    2020
1518    2020
1519    2020
1520    2020
1521    2020
1522    2020
1523    2020
1524    2020
1525    2020
Name: cvm_reg, dtype: object

In [20]:
ds.iloc[index[0], 41] = 1998

In [21]:
## Compute firm age again.
ds['idade_firma'] = ds['ano'].astype(int) - ds['cvm_reg'].astype(int)

In [22]:
## Checking negative values
ds.loc[ds['idade_firma'] < 0]


,setor_economatica,setor_bovespa,id_cvm,codigo,listagem_bovespa,at,k,ll,d_cp,d_lp,...,div_pl,roa,roe,rok,fcl_normalizado,divonerosa_normalizado,tx_vendas,epu,cvm_reg,idade_firma


In [24]:
## Exporting to continue in R
ds.to_csv(f'{module_path}/raw-data/ds_to_r.csv', index=False, header=True)